# Lab Week 6
One issue in natural language processing is the identification of sentiment of texts, are they positive, negative or neutral to some concept. This dataset (https://github.com/thanhtut/info284_lab/blob/master/assignment1/twitter-airline-sentiment/Tweets.csv) has a collection of tweets about airlines with corresponding sentiments (negative, positive, neutral).

The task :
The task we want you to do is to run three machine learning models using Naïve Bayes on the tweet texts.

1. Prepare the data for machine learning. Extract the relevant columns, do language preprocessing of the tweets, etc.
2. Run a BernoulliNB to classify sentiment
3. Run a MultiomialNB to classify sentiment
4. Run a MultinomialNB to classify sentiment, but insteadof using a count vector you should use a TF-IDF-vector for each instance. What is TF-IDF? Read sources on information retrieval, and also look here: https://stackabuse.com/text-classification-with-python-and-scikit-learn

How does these three approaches compare?

### What can we tell about the data?
The dtypes vary, none of the data has nulls, we´ll double check anyways, mostly objects witch will be timeforamts and strings.

We can analyse the data:

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset/Tweets.csv")

In [3]:
print("Shape of the dataframe is",df.shape)

Shape of the dataframe is (14640, 15)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [5]:
print("Percentage null or na values in tweets")
((df.isnull() | df.isna()).sum() * 100 / df.index.size).round(2)

Percentage null or na values in tweets


tweet_id                         0.00
airline_sentiment                0.00
airline_sentiment_confidence     0.00
negativereason                  37.31
negativereason_confidence       28.13
airline                          0.00
airline_sentiment_gold          99.73
name                             0.00
negativereason_gold             99.78
retweet_count                    0.00
text                             0.00
tweet_coord                     93.04
tweet_created                    0.00
tweet_location                  32.33
user_timezone                   32.92
dtype: float64

In [6]:
print(df.duplicated().any())
print(len(df))

True
14640


In [7]:
df = df.drop_duplicates()
print(len(df))

14604


### Choosing the necesarry columns:
After removing duplications and looking for emtpy columns we can pick out the colums that should be used.
`airline_sentiment_gold` , `negativereason_gold` and `tweet_coord` is all but empty so they will prove no help.
`tweet_id`, `name` and `tweet_created` are rather unique so they will not give anything to the table. The confidence columns are also uneccesary. That leaves us with the remaining:


In [8]:
tweets = df.drop(columns = [
    "airline_sentiment", "airline_sentiment_confidence", "negativereason_confidence",
    "tweet_id", "airline_sentiment_gold", "name", "negativereason_gold",
    "tweet_coord", "tweet_created", "tweet_location", "user_timezone"
])

In [9]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14604 entries, 0 to 14639
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   negativereason  9159 non-null   object
 1   airline         14604 non-null  object
 2   retweet_count   14604 non-null  int64 
 3   text            14604 non-null  object
dtypes: int64(1), object(3)
memory usage: 570.5+ KB


In [10]:
X = tweets["text"]
y = df["airline_sentiment"]

In [11]:
print(X.shape)
print(y.shape)
#y = y.to_numpy()
#y = y.reshape(y.shape[0],1)
print("-------------")
print(X.shape)
print(y.shape)

(14604,)
(14604,)
-------------
(14604,)
(14604,)


Lets Vectorize the data using `CountVectorizer` the sklearn function converts a collection of text documents to a matrix of token counts

In [12]:
vect = CountVectorizer()

X = vect.fit_transform(X)

Lets split the data into random train and test subsets and fit it to the `BernoulliNB()` model

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [14]:
model = BernoulliNB().fit(X_train, y_train)
p_train = model.predict(X_train)
p_test = model.predict(X_test)

Now let us calculate the accurasy and look at the results

In [15]:
acc_train = accuracy_score(y_train, p_train)
acc_test = accuracy_score(y_test, p_test)
print(f"Train Accuracy: {acc_train}\n Test Accuracy: {acc_test}")

Train Accuracy: 0.831826896740619
 Test Accuracy: 0.7619830183511367
